In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, svm, model_selection
from sklearn.linear_model import LogisticRegression


In [275]:
# Import data
#df = pd.read_csv('customer_data.csv')
df = pd.read_csv('Nordic Brain.csv')
df_match_test = df.drop(df_nonchurners_testgroup.index)

,customer_id,month,months_since_joined,referred_friends,transaction_count,transaction_value,clv,total_reward_points,earned_reward_points,cluster,churned
0,2,2020-11-01,107,0,103.0,1550.845992,51687.097864,260276.993260,7931.0,B,False
1,20,2020-11-01,107,0,77.0,1069.887835,35657.568520,144683.993745,4081.0,B,False
2,30,2020-11-01,107,0,98.0,98.000000,3266.175762,34817.597635,392.0,C,False
3,40,2020-11-01,107,2,20.0,139.242815,4640.729662,7539.996820,120.0,C,False
4,43,2020-11-01,107,1,8.0,120.234386,4007.210576,8442.021801,48.0,B,False
...,...,...,...,...,...,...,...,...,...,...,...
345240,1434882,2020-11-01,1,0,23.0,61.019996,289.238866,24.587028,69.0,B,True
345261,1434903,2020-11-01,1,0,27.0,78.286019,371.080975,0.942322,81.0,B,True
345265,1434907,2020-11-01,1,0,21.0,60.409446,286.344821,4.031503,63.0,C,True
345294,1434936,2020-11-01,1,1,20.0,50.783601,240.717667,39.999218,40.0,B,True


In [372]:
df_churners = df[df.churned == True]
df_nonchurners = df[df.churned == False]
df_nonchurners_testgroup = df_nonchurners.tail(15*len(df_churners))
df_nonchurners = df_nonchurners.drop(df_nonchurners_testgroup.index)
df_testgroup = df_churners.append(df_nonchurners_testgroup)
df_nonchurners

,customer_id,month,months_since_joined,referred_friends,transaction_count,transaction_value,clv,total_reward_points,earned_reward_points,cluster,churned
0,2,2020-11-01,107,0,103.0,1550.845992,51687.097864,260276.993260,7931.0,B,False
1,20,2020-11-01,107,0,77.0,1069.887835,35657.568520,144683.993745,4081.0,B,False
2,30,2020-11-01,107,0,98.0,98.000000,3266.175762,34817.597635,392.0,C,False
3,40,2020-11-01,107,2,20.0,139.242815,4640.729662,7539.996820,120.0,C,False
4,43,2020-11-01,107,1,8.0,120.234386,4007.210576,8442.021801,48.0,B,False
...,...,...,...,...,...,...,...,...,...,...,...
112477,883347,2020-11-01,35,0,44.0,288.636848,8008.907855,8826.999909,616.0,B,False
112478,883351,2020-11-01,35,0,32.0,258.550333,7174.086787,6234.999901,384.0,B,False
112479,883353,2020-11-01,35,0,24.0,130.685178,3626.167462,2897.994069,144.0,A,False
112480,883356,2020-11-01,35,1,45.0,45.000000,1248.630783,2031.904993,90.0,B,False


In [373]:
# Omitting customer id
df_testgroup = df_testgroup.iloc[:,1:]

In [374]:
# Replacing bools with 1 and 0. I guess they evaluate to 1 and 0 but I do it anyway
df_testgroup.churned.replace({True:1, False:0}, inplace = True)
df_nonchurners.churned.replace({True:1, False:0}, inplace = True)

In [375]:
dummy_df = pd.get_dummies(df_testgroup)
dummy_df2 = pd.get_dummies(df_nonchurners.iloc[:,1:])

In [376]:
y = dummy_df.churned.values
dummy_df2

,months_since_joined,referred_friends,transaction_count,transaction_value,clv,total_reward_points,earned_reward_points,month_2020-11-01,cluster_A,cluster_B,cluster_C,churned_0
0,107,0,103.0,1550.845992,51687.097864,260276.993260,7931.0,1,0,1,0,1
1,107,0,77.0,1069.887835,35657.568520,144683.993745,4081.0,1,0,1,0,1
2,107,0,98.0,98.000000,3266.175762,34817.597635,392.0,1,0,0,1,1
3,107,2,20.0,139.242815,4640.729662,7539.996820,120.0,1,0,0,1,1
4,107,1,8.0,120.234386,4007.210576,8442.021801,48.0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
112477,35,0,44.0,288.636848,8008.907855,8826.999909,616.0,1,0,1,0,1
112478,35,0,32.0,258.550333,7174.086787,6234.999901,384.0,1,0,1,0,1
112479,35,0,24.0,130.685178,3626.167462,2897.994069,144.0,1,1,0,0,1
112480,35,1,45.0,45.000000,1248.630783,2031.904993,90.0,1,0,1,0,1


In [377]:
X = dummy_df.drop('churned', axis = 1)
X_nonchurners = dummy_df2.drop('churned_0', axis = 1)

In [378]:
cols = X.columns

In [379]:
mm = preprocessing.MinMaxScaler()

In [380]:
X = pd.DataFrame(mm.fit_transform(X))
X_nonchurners = pd.DataFrame(mm.fit_transform(X_nonchurners))

In [381]:
logreg = LogisticRegression(fit_intercept = False, C = 1e12, solver 
= 'liblinear')
logreg.fit(X, y)

LogisticRegression(C=1000000000000.0, fit_intercept=False, solver='liblinear')

In [382]:
y_prediction = logreg.predict(X_nonchurners)
Y = logreg.predict_proba(X_nonchurners)
y_prediction

array([1, 1, 1, ..., 0, 1, 0])

In [383]:
# Find residual differences between train data and predicted train data
resiudals = np.abs(y_train, y_hat_train)
# Print the number of times our model was correct ('0') and incorrect ('1')
print(pd.Series(resiudals).value_counts())
# Print normalized amount of times our model was correct (percentage)
print(pd.Series(resiudals).value_counts(normalize = True))

0    66069
1    10869
dtype: int64
0    0.85873
1    0.14127
dtype: float64


In [384]:

dummy_df2['Prediction'] = y_prediction
dummy_df2['Probabilities'] = Y[:,1]

In [385]:
df_nonchurners['Probability'] = Y[:,1] 
df_nonchurners

,customer_id,month,months_since_joined,referred_friends,transaction_count,transaction_value,clv,total_reward_points,earned_reward_points,cluster,churned,Probability
0,2,2020-11-01,107,0,103.0,1550.845992,51687.097864,260276.993260,7931.0,B,0,1.000000
1,20,2020-11-01,107,0,77.0,1069.887835,35657.568520,144683.993745,4081.0,B,0,1.000000
2,30,2020-11-01,107,0,98.0,98.000000,3266.175762,34817.597635,392.0,C,0,0.999437
3,40,2020-11-01,107,2,20.0,139.242815,4640.729662,7539.996820,120.0,C,0,0.005609
4,43,2020-11-01,107,1,8.0,120.234386,4007.210576,8442.021801,48.0,B,0,0.020006
...,...,...,...,...,...,...,...,...,...,...,...,...
112477,883347,2020-11-01,35,0,44.0,288.636848,8008.907855,8826.999909,616.0,B,0,0.361923
112478,883351,2020-11-01,35,0,32.0,258.550333,7174.086787,6234.999901,384.0,B,0,0.360962
112479,883353,2020-11-01,35,0,24.0,130.685178,3626.167462,2897.994069,144.0,A,0,0.350841
112480,883356,2020-11-01,35,1,45.0,45.000000,1248.630783,2031.904993,90.0,B,0,0.555667


In [389]:

average_spend = df_nonchurners.transaction_value/df_nonchurners.transaction_count
clv = average_spend/df_nonchurners.Probability
df_nonchurners['New_clv'] = clv 
df_submit=df_nonchurners[df_nonchurners.New_clv > 50]
df_submit = df_submit.customer_id
df_submit.to_csv('Nordic_brain.csv', index = False)

In [387]:
print(len(df_submit))
df_nonchurners[df_nonchurners.New_clv => 50]

9911


,customer_id,month,months_since_joined,referred_friends,transaction_count,transaction_value,clv,total_reward_points,earned_reward_points,cluster,churned,Probability,New_clv
3,40,2020-11-01,107,2,20.0,139.242815,4640.729662,7539.996820,120.0,C,0,0.005609,1241.235927
4,43,2020-11-01,107,1,8.0,120.234386,4007.210576,8442.021801,48.0,B,0,0.020006,751.249122
15,214,2020-11-01,107,2,12.0,182.554534,6084.236686,9837.893597,108.0,B,0,0.019288,788.739702
38,518,2020-11-01,107,0,12.0,157.222117,5239.949674,8214.999470,84.0,B,0,0.016815,779.160664
70,914,2020-11-01,107,2,13.0,98.596989,3286.072396,1436.632513,52.0,C,0,0.000459,16517.412647
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112440,883158,2020-11-01,35,1,39.0,340.003960,9434.209142,8229.935409,663.0,C,0,0.123047,70.851232
112446,883166,2020-11-01,35,3,33.0,263.018330,7298.061859,5312.999820,429.0,C,0,0.102340,77.880142
112450,883183,2020-11-01,35,1,42.0,383.814309,10649.830251,10513.502643,798.0,B,0,0.174703,52.308288
112455,883226,2020-11-01,35,2,29.0,303.525527,8422.029267,6161.998919,435.0,C,0,0.129999,80.511112


In [388]:
average_spend = df.transaction_value/df.transaction_count

In [136]:
df

,customer_id,month,months_since_joined,referred_friends,transaction_count,transaction_value,clv,total_reward_points,earned_reward_points,cluster,churned,churn_prob,Churn_probabillity_x,Churn_probabillity_y
14,112,2017-02-01,62,1,52.0,369.177090,12141.632321,24939.908763,936.0,B,False,0.986978,0.986978,0.986978
20,154,2017-02-01,62,2,41.0,433.931634,14271.303656,17871.986812,861.0,A,False,0.995827,0.995827,0.995827
21,158,2017-02-01,62,1,51.0,264.230567,8690.112363,14229.261027,663.0,C,False,0.990719,0.990719,0.990719
22,168,2017-02-01,62,1,40.0,250.752496,8246.840578,15552.560531,480.0,C,False,0.987426,0.987426,0.987426
25,204,2017-02-01,62,0,56.0,654.573230,21527.845863,35533.999369,1792.0,B,False,0.994100,0.994100,0.994100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13131231,1423188,2020-10-01,1,2,15.0,43.892955,208.055547,29.999991,30.0,B,False,0.881360,0.881360,0.881360
13131233,1423190,2020-10-01,1,1,27.0,72.793022,345.043798,80.999991,81.0,B,False,0.939575,0.939575,0.939575
13131238,1423195,2020-10-01,1,0,20.0,54.178116,256.807897,39.999639,40.0,B,False,0.861103,0.861103,0.861103
13131239,1423196,2020-10-01,1,1,17.0,52.367784,248.226802,33.999996,34.0,B,False,0.885098,0.885098,0.885098


345316